In [43]:
import math
import pickle
import scipy.io as sio
from scipy.spatial.distance import cdist
from FeatureExtractor import *
from config_voting import *
category='car'
set_type='test'
file_nm = 'bbox_props_{0}_{1}_{2}_2.mat'.format(category, dataset_suffix, set_type)
# file_nm='test.mat'
file_bbox_proposals = os.path.join(Data['root_dir'], file_nm)

assert(os.path.isfile(file_bbox_proposals))
mat_contents = sio.loadmat(file_bbox_proposals)
Box = mat_contents['Box']
if category=='car':
    print(Box['anno'][0,50]['height'][0,0][0,0])
    print(Box['boxes'][0,50].shape)
else:
    print(Box[50,0]['name'][0,0][0])
    print(Box[50,0]['boxes'][0,0].shape)
# mat_contents['Box']['anno'][0,50]['height'][0,0][0,0]
# mat_contents['Box']['anno'][0,50]['height'][0,0][0,0]
# mat_contents['Box'].shape

334
(217, 5)


In [48]:
import glob

glob.glob('/mnt/4T-HD/qing/intermediate/feat/props_feat_car_mergelist_rand_test_*.pickle')

['/mnt/4T-HD/qing/intermediate/feat/props_feat_car_mergelist_rand_test_1.pickle',
 '/mnt/4T-HD/qing/intermediate/feat/props_feat_car_mergelist_rand_test_5.pickle',
 '/mnt/4T-HD/qing/intermediate/feat/props_feat_car_mergelist_rand_test_4.pickle',
 '/mnt/4T-HD/qing/intermediate/feat/props_feat_car_mergelist_rand_test_6.pickle',
 '/mnt/4T-HD/qing/intermediate/feat/props_feat_car_mergelist_rand_test_3.pickle',
 '/mnt/4T-HD/qing/intermediate/feat/props_feat_car_mergelist_rand_test_0.pickle',
 '/mnt/4T-HD/qing/intermediate/feat/props_feat_car_mergelist_rand_test_7.pickle',
 '/mnt/4T-HD/qing/intermediate/feat/props_feat_car_mergelist_rand_test_2.pickle']

In [ ]:
category='car'
set_type='train'
file_cache_feat = os.path.join(Feat['cache_dir'], '{0}_{1}_{2}.pickle'.format(category, dataset_suffix, set_type))
assert(os.path.isfile(file_cache_feat))
with open(file_cache_feat, 'rb') as fh:
    feat_set = pickle.load(fh)
    
assert(os.path.isfile(Dictionary))
with open(Dictionary, 'rb') as fh:
    _,centers = pickle.load(fh)
     
assert(centers.shape[0]==VC['num'])
print('compute and cache VC distance data for {1} {0} set ...\n'.format(set_type, category))

In [3]:
r_set = [None for nn in range(len(feat_set))]
for nn in range(1):
    layer_feature = feat_set[nn]
    print(layer_feature.shape)
    height = layer_feature.shape[0]
    width = layer_feature.shape[1]
    assert(featDim == layer_feature.shape[2])
    
    layer_feature = layer_feature.reshape(-1, featDim)
    feat_norm = np.sqrt(np.sum(layer_feature**2, 1)).reshape(-1,1)
    layer_feature = layer_feature/feat_norm
    
    dist = cdist(layer_feature, centers).reshape(height,width,-1)
    print(dist.shape)
    assert(dist.shape[2]==centers.shape[0]);
    r_set[nn] = dist


(14, 14, 512)
(14, 14, 198)
